<a href="https://colab.research.google.com/github/mwelland/ENGPYHS_3NM4/blob/main/Linear_systems_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Goals
- Describe the computational complexity of algorithms
- Formalize how we solve linear systems
- Show how to transform general linear systems into simpler triangular systems with Gauss Elimination
- Introduce LU decomposition

# Computational complexity
The *computational complexity* of an algorithm characterizes the number of operations it requires (thus comparing the algorithm instead of the hardware). Linear algebra algorithms are characterized in terms of the dimension of their arguments. Complexity is represented in big O notation (similar to the accuracy of function approximations). I.e.: Only the leading order is kept, and constants disregarded.



###Example
Traditional matrix multiplication of two $n \times n$ matricies - involves $n^3$ operations and is thus $O(n^3)$

NB: This is algorithm-dependant - The Strassen algorithm has $O^{2.81}$

Some complexities of common linear algebra operations:

| Operation | Description | Complexity |
|---|---|---|
| Matrix addition | Adding two matrices of the same size | O(n²) |
| Matrix multiplication | Multiplying two matrices | O(n³) (standard algorithm) |
| Matrix-vector multiplication | Multiplying a matrix by a vector | O(n²) |
| Matrix inversion | Finding the inverse of a matrix | O(n³) |
| Determinant calculation | Computing the determinant of a matrix | O(n³) |
| Transpose | Swapping rows and columns of a matrix | O(n²) |

High-performance computing introduces a new element to complexity which deals with how algorithms parallelize. This is called *scaling* and measured in $O(number \ of \ nodes)$. The goal is usually $O(n)$ but is hard to achieve!

#Algorithms for solving linear systems

Gauss elimination is an algorithm for the most familiar / intuitive solution technique. Let's reexamine our analytical (symbolic) solution to the previous problem and then make it into a numerical algorithm.

Question:
You are organizing a fundraising event and need to buy chairs and tables. Chairs cost \$20 each and tables cost \$50 each. You have a budget of \$700 and need a total of 20 pieces of furniture (chairs and tables combined). How many chairs and tables should you buy?

## Symbolic manipulation

Let $c$ and $t$ be the number of chairs and tables respectively.

The budget and pieces equations are,

(1) $20 c + 50 t = 700$

(2) $  c+t = 20$

Solve (2) for $c$:

$c = 20-t$

Substitute into (1):

$20 [20-t] + 50t = 700$

$t = 10$

and substitute into (2) or (1) to find $c$.

This works because our first step carries the unkown *symbol* $t$.

## Numerical approach

Lets repeat this without carrying symbols.

(1) $20 c + 50 t = 700$

(2) $  c+t = 20$

Multiply (2) by $20$:

$20 c + 20 t = 400$

and subtract from (1):

\begin{eqnarray*}
20 c + &50 t  &=& 700 \\
-20 c - &20 t &=& -400 \\
\hline
 &30 t &=& 300
\end{eqnarray*}

Thus we have simplified the last line until it reaches a trival solution for $t$. Now it is a matter of *substitution* to solve for $c$.


NB: the linear system has been changed (without changing the answer) to:

$\begin{pmatrix}
20 & 50 \\
0 & 30
\end{pmatrix}
\begin{pmatrix}
c \\
t
\end{pmatrix} =
\begin{pmatrix}
700 \\
300
\end{pmatrix}$

In particular, $A$ is *upper triangular*, from which the answer is easily obtained through *backward substitution*.

##Upper triangular matricies

An upper triangular matrix, $U$, is a matrix whose elements below the diagonal are zero:

$$\begin{bmatrix}
u_{1,1} & u_{1,2} & u_{1,3} & u_{1,4}\\
0 & u_{2,2}' & u_{2,3}' & u_{2,4}'\\
0 & 0 & u_{3,3}' & u_{3,4}' \\
0 & 0 & 0 & u_{4,4}'
\end{bmatrix}$$

This is useful because, in equation form,

$U x = b'$

turns in to
\begin{eqnarray*}
\begin{array}{}
 u_{1,1} x_1 &+& u_{1,2} x_2 & + & u_{1,3} x_{3} &+&u_{1,4} x_4 &=& b'_1,\\
& & u_{2,2}' x_{2} &+ & u_{2,3}' x_{3} &+& u_{2,4}' x_4 &=& b_{2}' \\
&& & & u_{3,3}' x_{3} &+& u_{3,4}' x_4 &=& b_{3}',\\
&& && && u_{4,4}' x_4 &=& b_{4}'.
\end{array}
\end{eqnarray*}

##Gauss Elimination

Gauss Elimination proceeds in two phases. The first is elimination which transforms

$A x = b$

into

$U x = b'$

where $U$ is an upper triangular matrix and $b'$ is a modified vector of constants. The second phase is back-substitution which is trivial with triangular matricies.

### Gauss elimination algorithm

We first take

$Ax = b$

and build an *Augmented matrix*:

$Ab = [ A | b ]$

 We are allowed the following operations which affect $|A|$ but not the solution to the problem:

|Operation | Effect on $|A|$ |
|-----|-----|
|Exchange 2 rows | Flips sign|
|Multiply a row by § | Multiplied by § |
|Subtract 2 rows     | Unchanged|

###Steps from our previous example

In [ ]:
import numpy as np
# Define A and b

A = np.array([[20, 50],
              [1, 1]])

b = np.array([[700], [20]])

#Form the augmented matrix A|b
Ab = np.hstack([A, b])

#The coefficient matrix can be separated by slicing Ab
def print_update():
  print("Augmented matrix is \n", Ab)
  print("Determinant of A: ", np.linalg.det(Ab[:,:-1])),
  print("Norm of A", np.linalg.norm(Ab[:,:-1], 'fro')),
  print("Condition number of A:", np.linalg.cond(Ab[:, :-1]))
  print("\n")

print("Step 0: Show the augmented matrix and the determinant of A")
print_update()

print("Step 1: Multiply the second row by 20")
Ab[1, :] = Ab[1, :] * 20
print_update()

print("Step 2: Subtract the second row from the first")
Ab[1, :] = Ab[1, :] - Ab[0, :]
print_update()

Note: The determinant, norm and condition number are all changing despite the answer remaining the same. This is the basis of preconditioners!

### Algorithm



Let's describe an algorithm for Gaussian elimination, as a sequence of passes row by row through the matrix.

Each pass we choose a *pivot row* which is used to eliminate the elements in other equations through multiplication of the pivot and subtraction.

Start from the top and only pass downwards, so we end up with an upper triangular matrix.

####Example

Use Gauss Elimination to solve the following equations.

\begin{eqnarray*}
4x_1 + 3x_2 - 5x_3 &=& 2 \\
-2x_1 - 4x_2 + 5x_3 &=& 5 \\
8x_1 + 8x_2  &=& -3 \\
\end{eqnarray*}

Step 1: Turn these equations to matrix form $Ax=y$.

$$
\begin{bmatrix}
4 & 3 & -5\\
-2 & -4 & 5\\
8 & 8 & 0\\
\end{bmatrix}\left[\begin{array}{c} x_1 \\x_2 \\x_3 \end{array}\right] =
\left[\begin{array}{c} 2 \\5 \\-3\end{array}\right]$$

Step 2: Get the augmented matrix [A, y]

$$
[A, y]  = \begin{bmatrix}
4 & 3 & -5 & 2\\
-2 & -4 & 5 & 5\\
8 & 8 & 0 & -3\\
\end{bmatrix}$$

Step 3: Choose the first equation as the pivot equation and turn the 2nd row first element to 0. To do this, we can multiply -0.5 for the 1st row (pivot equation) and subtract it from the 2nd row. The multiplier is $m_{2,1}=-0.5$. We will get

$$
\begin{bmatrix}
4 & 3 & -5 & 2\\
0 & -2.5 & 2.5 & 6\\
8 & 8 & 0 & -3\\
\end{bmatrix}$$

Step 4: Turn the 3rd row first element to 0. We can do something similar, multiply 2 to the 1st row and subtract it from the 3rd row. The multiplier is $m_{3,1}=2$. We will get

$$
\begin{bmatrix}
4 & 3 & -5 & 2\\
0 & -2.5 & 2.5 & 6\\
0 & 2 & 10 & -7\\
\end{bmatrix}$$

Step 5: Turn the 3rd row 2nd element to 0. We can multiple -4/5 for the 2nd row, and subtract it from the 3rd row. The multiplier is $m_{3,2}=-0.8$. We will get

$$
\begin{bmatrix}
4 & 3 & -5 & 2\\
0 & -2.5 & 2.5 & 6\\
0 & 0 & 12 & -2.2\\
\end{bmatrix}$$

Elmination is now complete since $A$ is upper triangular.

Proceed with substitution:

Step 6: Therefore, we can get $x_3=-2.2/12=-0.183$.

Step 7: Insert $x_3$ to the 2nd equation, we get $x_2=-2.583$

Step 8: Insert $x_2$ and $x_3$ to the first equation, we have $x_1=2.208$.

### Complexity

Considering the two phases of Gauss elimination:

Elimination: ~$n^3/3$ operations, therefore, $O(n^3)$

Back substitution: $O(n^2/2)$


##Gauss-Jordan Elimination
An obvious extension is to conduct each pass both upwards and downwards (G.E. is just down). While doing this we also normalize the row to get reduced row echelon form:

$$
\begin{bmatrix}
1 & 0 & 0 & 2.208\\
0 & 1 & 0 & -2.583\\
0 & 0 & 1 & -0.183\\
\end{bmatrix}$$

Since $I^{-1} = I$, the answer is just the right hand vector.

### Complexity
Gauss-Jordan Elimination eliminates the need for back-substitution so one might think that it is more efficient than Gauss Elimination. Unfortunatley, the elimination phase takes ~$~n^3/2$ operations whereas GE took ~$n^3/3 + n^2/2$. therefore Gauss Elimination is preferred.

#Pivoting and diagonal dominance



The order of equations, and therefore rows in the matrix, is obviously arbitrary, but this may break Gauss Elimination.

E.g.: a system with augmented matrix

$$
\begin{bmatrix}
2 & -1 & 0 & 1\\
-1 & 2 & -1 & 0\\
0 & -1 & 1 & 0\\
\end{bmatrix}$$

will work with our GE scheme, but the same system reordered,

$$
\begin{bmatrix}
0 & -1 & 1 & 0\\
-1 & 2 & -1 & 0\\
2 & -1 & 0 & 1\\
\end{bmatrix}$$

will fail due to the $0$ in the element of the first row.

More generally, if the diagonal element is *small* we will encounter roundoff error:

$$
\begin{bmatrix}
\epsilon & -1 & 1 & 0\\
-1 & 2 & -1 & 0\\
2 & -1 & 0 & 1\\
\end{bmatrix}$$

after the first pass leads to:

$$
\begin{bmatrix}
\epsilon & -1 & 1 & 0\\
-1 & 2 -1/\epsilon & -1 + 1/\epsilon & 0\\
2 & -1 -1/\epsilon& + 1/\epsilon & 1\\
\end{bmatrix}$$

from which we immediately see the danger as $\epsilon \rightarrow 0$!

Thus it may be important to swap rows in order to have GE succeed. In particular you don't want the diagonal to have $0$s or *small* numbers...


##Diagonal dominance

A matrix is said to be *diagonally dominant* if each diagonal is larger, in absolute value, than the *sum of the other elements in the row*.

$|A_{ii}| \ge \sum_{j=1,j \ne i}^{n} |A_{ij}|$


Example: Which is diagonally dominant?

a) $$
\begin{bmatrix}
-2 & 4 & -1 \\
-1 & -1 & 3 \\
4 & -2 & 1 \\
\end{bmatrix}$$

b)
$$
\begin{bmatrix}
4 & -2 & 1 \\
-2 & 4 & -1 \\
-1 & -1 & 3 \\
\end{bmatrix}$$

##Importance of diagonal dominance
It can be shown that if a matrix is diagonally dominant, *it will not benefit from pivoting!*.

As a side benefit, it precludes singular matricies.  

Diagonal dominance contributes generaelly to numerical stability and accuracy through minimizing roundoff error propogation. This result extends beyond Gauss elimination, and will also be a criteria for matrix factorization methods and the efficient convergence of iterative methods.